In [ ]:
%load_ext autoreload
%autoreload 2
import sys
from ridge_utils.DataSequence import DataSequence
import pandas as pd
import os
import matplotlib.pyplot as plt
import cortex
import seaborn as sns
from os.path import join
from collections import defaultdict
import numpy as np
import sasc.viz
import joblib
import dvu
import sys
sys.path.append('../notebooks')
from tqdm import tqdm
from sasc.config import FMRI_DIR, STORIES_DIR, RESULTS_DIR
from neuro.config import repo_dir
from neuro import analyze_helper, viz
from neuro.features.qa_questions import get_questions, get_merged_questions_v3_boostexamples
flatmaps_per_question = __import__('06_flatmaps_per_question')
import viz

Note, this notebook requires first running `03_export_qa_flatmaps.ipynb` into `df_qa_dict.pkl` files for each subject.

### load gemv average flatmaps

In [ ]:
gemv_avgs_pilot = joblib.load(join(
    RESULTS_DIR, "processed", "flatmaps", 'resps_avg_dict_pilot.pkl'))
gemv_avgs_pilot5 = joblib.load(join(
    RESULTS_DIR, "processed", "flatmaps", 'resps_avg_dict_pilot5.pkl'))
gemv_avg_flatmaps = gemv_avgs_pilot | gemv_avgs_pilot5

# Make comparison

In [ ]:
# matches
qa_list = [
    # approx matches
    'Is time mentioned in the input?',
    'Does the input contain a measurement?',
    # 'Does the input contain a number?',
    'Does the sentence mention a specific location?',
    # 'Does the sentence describe a relationship between people?',
    # 'Does the sentence describe a relationship between people?',
    'Does the text describe a mode of communication?',
    # 'Does the sentence contain a negation?',
]
gemv_list = [
    # approx matches
    ('time', 212),
    ('measurements', 171),
    # ('measurements', 171),
    # ('moments',	337),
    # ('locations', 122),
    ('locations', 368),
    # ('emotion', 179),
    # ('emotional expression', 398),
    ('communication', 299),
    # ('negativity', 248)
]


qa_list += [
    'Is the sentence abstract rather than concrete?',
    'Does the sentence contain a cultural reference?',
    'Does the sentence include dialogue?',
    'Is the input related to a specific industry or profession?',
    'Does the sentence contain a negation?',
    'Does the input contain a number?',
    "Does the sentence express the narrator's opinion or judgment about an event or character?",
    'Does the sentence describe a personal or social interaction that leads to a change or revelation?',
    'Does the sentence describe a personal reflection or thought?',
    'Does the sentence involve an expression of personal values or beliefs?',
    'Does the sentence describe a physical action?',
    'Does the input involve planning or organizing?',
    'Does the sentence contain a proper noun?',
    'Does the sentence describe a relationship between people?',
    'Does the sentence describe a sensory experience?',
    'Does the sentence involve the mention of a specific object or item?',
    'Does the sentence include technical or specialized terminology?',
]

gemv_list += [
    ('abstract descriptions', 'qa'),
    ('cultural references', 'qa'),
    ('dialogue', 'qa'),
    ('industry or profession', 'qa'),
    ('negations', 'qa'),
    ('numbers', 'qa'),
    ('opinions or judgments', 'qa'),
    ('personal or interactions interactions', 'qa'),
    ('personal reflections or thoughts', 'qa'),
    ('personal values or beliefs', 'qa'),
    ('physical actions', 'qa'),
    ('planning or organizing', 'qa'),
    ('proper nouns', 'qa'),
    ('relationships between people', 'qa'),
    ('sensory experiences', 'qa'),
    ('specific objects or items', 'qa'),
    ('technical or specialized terminology', 'qa')
]

df_pairs = pd.DataFrame({
    'qa_weight': qa_list,
    'gemv_avg_resp': gemv_list,
})

In [ ]:
subject = 'S02'
df_qa_dict = joblib.load(f'df_qa_dict_{subject.replace("UT", "")}.pkl')

In [ ]:
# for setting in df_qa_dict.keys():
for setting in ['shapley35', 'avg_individual']:
    df_qa_weights = df_qa_dict[setting]

    if isinstance(df_qa_weights, pd.DataFrame):
        df_pairs = df_pairs[df_pairs['qa_weight'].isin(df_qa_weights.index)]
        flatmaps_qa = df_qa_weights.loc[df_pairs['qa_weight'].values]['weights'].values
    else:
        flatmaps_qa = [df_qa_weights[bd]
                       for bd in df_pairs['qa_weight'].values]
    flatmaps_gemv = [gemv_avg_flatmaps[bd]
                     for bd in df_pairs['gemv_avg_resp'].values]

    corrs = viz._calc_corrs(
        flatmaps_qa,
        flatmaps_gemv,
        titles_qa=[analyze_helper.abbrev_question(
            q) for q in df_pairs['qa_weight'].astype(str)],
        titles_gt=df_pairs['gemv_avg_resp'].apply(lambda x: x[0]).astype(str)
    )

    viz.corr_bars(
        corrs,
        out_dir_save=join(repo_dir, 'qa_results', 'hypothesis_tests', setting),
        xlab='GEM-V',
    )
    plt.show()